<a href="https://colab.research.google.com/github/Sanjrani59/Anwar_Iqbal-450563-AI_Assignment2/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**bold text** Libraries

In [2]:
from collections import OrderedDict
import numpy as np
import torch
from torch import optim
import torch.nn as nn
import torchvision
from torchvision import transforms,models
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
import os

In [3]:

# Clean directories first
!rm -rf '/content/sample_data/transferlearning/train/.ipynb_checkpoints'
!rm -rf '/content/sample_data/transferlearning/test/.ipynb_checkpoints'

# Define transforms
transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

# Verify paths exist
train_dir = '/content/sample_data/transferlearning/train'
test_dir = '/content/sample_data/transferlearning/test'

# Add these checks
assert os.path.exists(train_dir), f"Train directory {train_dir} not found!"
assert os.path.exists(test_dir), f"Test directory {test_dir} not found!"

# Create datasets
train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=transform)
test_dataset = torchvision.datasets.ImageFolder(test_dir, transform=transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)


AssertionError: Train directory /content/sample_data/transferlearning/train not found!

In [ ]:
def imshow(image_torch):
    # flip image channels to RGB
    image_torch = image_torch.numpy().transpose((1, 2, 0))
    plt.figure()
    plt.imshow(image_torch)

X_train, y_train = next(iter(train_loader))

# Make a grid from batch
image_grid = torchvision.utils.make_grid(X_train[:16, :, :, :], scale_each= True, nrow=4)

imshow(image_grid)

In [ ]:
model = models.densenet121(pretrained = True)
model

In [6]:
# freeze all layers
for params in model.parameters():
    params.requires_grad = False

In [7]:
model.classifier = nn.Sequential(OrderedDict([
    ('fc1',nn.Linear(1024,1)),
    ('Output',nn.Sigmoid())
]))
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [9]:
opt = optim.Adam(model.classifier.parameters())
loss_function = nn.BCELoss()
train_losses=[]

model.train()
NUM_EPOCHS = 10
for epoch in range(NUM_EPOCHS):
    train_loss= 0
    test_loss= 0
    for bat,(img,label) in enumerate(train_loader):

        # zero the gradients
        opt.zero_grad()

        # forward pass
        output = model(img)

        # calc losses
        loss = loss_function(output.squeeze(),label.float())

        # propagate losses
        loss.backward()

        # update weights
        opt.step()

        # update current train loss
        train_loss += loss.item()


    train_losses.append(train_loss)
    print(f"epoch: {epoch}, train_loss: {train_loss}")

epoch: 0, train_loss: 1.011156976222992
epoch: 1, train_loss: 0.882696807384491
epoch: 2, train_loss: 0.8069905936717987
epoch: 3, train_loss: 0.7220125794410706
epoch: 4, train_loss: 0.6571906208992004
epoch: 5, train_loss: 0.5917965769767761
epoch: 6, train_loss: 0.5461183488368988
epoch: 7, train_loss: 0.49946996569633484
epoch: 8, train_loss: 0.46018823981285095
epoch: 9, train_loss: 0.44798967242240906


In [1]:
sns.lineplot(x = range(len(train_losses)), y = train_losses)


NameError: name 'sns' is not defined